<style>
r { color: Red }
o { color: Orange }
g { color: Green }
b { color: LightBlue}
w { color: White}
</style>

# **Mobile Robotics Presentation Group 16**
EPFL Course MICRO-452, fall 2023
- Gilles Regamey 296642
- Julien Droulet 328082
- Tom Rathjens   325155
- Aubin Sabatier 310741

Video of the project:

<video width="500" height="350" controls src="demo.mp4">animation</video>

## Table of content
-   [Choices](#choices)
    - environement
    - filter
    - navigation
-   [Modules](#modules)
    - [Vision](#vision)
    - [Filtering](#filtering)
    - [Global Navigation](#globnav)
    - [Local Navigation](#locnav)
    - [Motion Control](#motion)
-   [Demonstration](#demo)
    - setup
    - loop
-   [Results](#results)



<a id="choices"></a>
# Choices taken
The project called for a environement map to navigate with fixed and un-planned obstacles to a destination point from any point. Fixed obstacles are constant trough the navigation and the robot can't use its sensor to detect them, they are only given by the camera at the start. Un-planned obstacles are detected only by the robot and can move at any time during navigation. 

## Environement
For the sake of simplicity, our map is a white A0 paper sheet with 4 <b>blue</b> box (4x4cm) at the corners. The Fixed obstacles are black pieces of cardboard of any shape. Mobile obstacles are <w>white</w> boxes of 4x4x4 cm 3d printed in PLA to have a more consistent reading with the proximity sensors. The robot is fitted with a paper hat with <r>red</r> circles in a isosceles triangle pattern for position and orientation. The destination is a big green circle 8cm in diameter. The image of the map is reframed to have a resolution of $1 \frac{px}{mm}$.

Below a theoretical map image before processing.

<img src="src\Vision\test_data\test_map.png" />

The fixed obstacles are abstracted in two ways for two different purposes:
- For Global navigation: occupancy grid of constant size.
- For Local navigation: polygonal contour approximation.
We'll see why in the modules descriptions.  

## Filtering
For the robot to reach its destination, it needs to know the path it has to take, as well as its current position in the event of new obstacles appearing or the destination changing during the mission. To achieve this, we use an external camera, positioned above the terrain, and speed sensors on the Thymio robot.

The camera gives us a good estimate of the robot's position and orientation. But the Thymio must be able to navigate without it if the robot goes out of its field of vision, if an obstacle blocks its field of vision, or if the camera fails to provide usable information due to lack of lighting. In addition, the sampling frequency of the position given by the camera is lower than the frequency of the measurements taken by the speed sensors.

However, speed sensor measurements are very noisy. The Kalman Filter algorithm compensates for the drawbacks of each method by merging the position estimate given by the camera with the speed sensor measurements.

## Global navigation

For the path planning we choose the A\* Algorithm with the occupancy grid given by the vision module. We choosed to use A\* as it is the simplest algorithm to have the optimal path
To have a margin to the obtacle we modified the cost of the cell by multipliying it by 2 if the cell is in a radius of 3 cells from an obstacle and by 1.5 if the cell is in a radius of 5 cell from the obstacle.

We also used the Douglas Peucker algorithm to simplify the path given by A\*, and have a smoother trajectory. 
Finally, we decided to use a path containing only the points where the direction is changing.

## Local navigation

Mobile and un-planned obstacles must be detected and avoided. This is done in `local_navigation()` It's called and executed in the main, when the front proximity sensors detect an obstacle `(prox value>500)`.

We choose to use a neural network system to be more robust to non-standard obstacles. In our testing, this system responded better to various obstacle shapes and orientations when compared to our second algorithm, the wall-following prototype. The limitations of the wall-following algorithm are mostly due to the absence of lateral sensors on the Thymio robot. The ANN is inspired by the course of Pr.Mondada.

The neural network system utilizes two sets of five weights to determine the target speeds for the left and right motors based on readings from each proximity sensor. To incorporate a global perspective of obstacles into the artificial neural network, we hallucinate the fixed obstacles with a custom function. This virtual sensor simulates obstacles by assigning proximity sensor values based on the robot's position and the global obstacle positions. For more details, checkout out Vision’s documentation.
The weights are tuned by repetitive testing.
We also implemented the `front_obst_avoidance_seq()` function. It's executed in the main program when the front proximity sensor detects an obstacle within 2 cm of the robot. This may occur if local obstacles are placed too late in front of the robot, leaving insufficient time for the ANN to navigate around them. In this case, the robot stops its motors, executes a $\pi$ turn, moves forward a specified distance, and then performs a -$\pi$ turn to resume its trip.

<a id="modules"></a>
# **Modules description**

<a id="vision"></a>
## **Vision**

The vision module is responsible for everything concerning the camera and vizualization.
Most importantly it has to detect the robot, the obstacles, the destination and the map correctly.
Visualization functions have been more of a tool to see and debug outputs of detection.
As our map elements are color based, we use the HLS (Hue Luminosity Saturation) encoding of the image to have a better chance at detecting color patches.
We use the opencv library extensively because of the blazingly fast parallel computation and all the usefull tools for detection and image processing.

### map detection
In the setup phase, we need to calibrate the camera to reframe the map in a consitant way to give a baseline for other functions. Opencv has a very usefull function called `warpPerspective()` just for that, we just need to give it the <w>warp matrix</w> and desired output shape. The function `vision.get_warp()` gives us this <w>warp matrix</w> from the image using the algorithm below and with the help of smaller functions called `reorder_border()`, `blob_point_list()` and another opencv function to compute the matrix from reference points `getPerspectiveTransform`. Note that this is not functionnal code, the real function and subsequent vision function can be found in `/src/vision.py`.
<small>

```python
def get_warp(capture_stream,ROI,padding,number_of_samples):
    while valid_samples < number_of_samples:
        frame = capture_stream.read()
        hsl_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HLS_FULL)
        nb_points,point_list = blob_point_list(Blue,CORNER_BLOB_FILT,CORNER_BLOB_VAL)
        if nb_points == 4:
            isvalid,ord_list = reorder_border(point_list)
            if isvalid:
                corners[:,:,smp] = ord_list
                valid_samples += 1
    avgCorn = mean(corners) 
     
    return cv2.getPerspectiveTransform(avgCorn,Sheetpts)
```

</small>

The `reorder_border()` function is just ordering the points relative to the center of the polygon they form (above/under,left/right) in order to give a consistant list of point for the averaging process.

Note that no action is taken for tangential and radial distortions. Opencv provides function for camera calibration and undistortion methods that need a calibration pattern (Charuco boards). We tried to use it but coulnd't get consitant camera matrix, it just made things worse most of the time so we decided to drop the idea knowing that robot position detection was good enough at our scale.

### Obstacle detection and abstraction

For the grid map of the obstacles we just apply a threshold to a scaled down filtered grayscale image of the map using opencv functions. You can also add a safe radius around obstacles using dilatation. 

<small>

```python
def get_grid_fixed_map(frame,shape,tresh=50,kernsz=5,robrad=80):
    kernel = np.ones((kernsz,kernsz),np.uint8)
    pxmap = cv2.inRange(frame,(0,0,0),(tresh,tresh,tresh))
    pxmap = cv2.morphologyEx(pxmap,cv2.MORPH_OPEN,kernel)
    if robrad != 0:
        safecircle = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(robrad,robrad))
        pxmap = cv2.dilate(pxmap,safecircle)
    temp = cv2.resize(pxmap, shape, interpolation=cv2.INTER_LINEAR)
    _,output = cv2.threshold(temp,10,1,type=cv2.THRESH_BINARY)
```
</small>

For the polygonal abstraction of the obstacles, we use opencv `findContours()` and `approxPolyDP()` to get an array of obstacles described as arrays of corner points.

<small>

```python
def get_obstacles(frame,tresh=50,eps=10,robrad=0):
    kernel = np.ones((5,5),np.uint8)
    pxmap = cv2.inRange(frame,(0,0,0),(tresh,tresh,tresh))
    pxmap = cv2.morphologyEx(pxmap,cv2.MORPH_OPEN,kernel)
    if robrad != 0:
        safecircle = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(robrad,robrad))
        pxmap = cv2.dilate(pxmap,safecircle)
    contp,hier =  cv2.findContours(pxmap,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    obstacles = []
    for cont in contp:
        cont = np.array(cont,dtype=np.float32).reshape((-1,2))
        scont = cv2.approxPolyDP(cont,eps,True)
        obstacles.append(scont)
    return obstacles
```
</small>

### Robot detection of position and orientation

This function is a bit more involved. The goal was to detect quickly the position between the wheels and the orientation vector. If we have a pattern with no rotational symetry, in our case a triangle with two side equal and longer than the last, we can get the points at the corners easily and compute everything with them. The hard part is knowing which point is which. Here is the algorithm:
<small>
```python
nbdot,points = filter_for_dots(hlsframe)
if nbdot >= 3:
    for p1 in range(0,nbdot-1):
        for p2 in range(p1+1,nbdot):
            pairwise_dist = dist(p1,p2)
else: 
    return False

max = argmax(pairwise_dist)
long = logic_array(abs(max-pairwise_dist)<epsilon)
short = logic_array(abs(max*ratio-pairwise_dist)<epsilon)
if(sum(short)==1 and sum(long)==2): 
    pairAB = where(shortidx == 1)
    Cidx = where((sum(longidx+longidx^t,1))==2)
    center = np.array([int((ptlist[pairAB[0][0]][0] + ptlist[pairAB[1][0]][0])/2),
                        int((ptlist[pairAB[0][0]][1] + ptlist[pairAB[1][0]][1])/2)])
    scale = d_table[pairAB]/4
    ptC = np.array([int(ptlist[Cidx[0][0]][0]),int(ptlist[Cidx[0][0]][1])])
    dirvect = ptC - center
    orient = -np.arctan2(dirvect[1],dirvect[0])
    return True,center,orient,scale
else:
    return False

```
<small>


<a id="filtering"></a>
## **Filtering**

The Extended Kalman Filter is an extension of the Kalman Filter for non-linear systems. The filter estimates the value of the vector (x, y, \theta, v, w), whose horizontal and vertical position depends on sinusoidal functions. To calculate the evolution of the position, the time interval between two measurements must take into account the time it takes for the program to be run. The time variable "dt" is initialized at the start of the program, then re-initialized at the end of a Kalman filter estimate. It must also be re-initialized if the thymio is moved during the mission.

Translational speed and angular velocity were calculated from sensor measurements on each wheel in the "speed_estimation" function. As the sensor measurements were not in standardized units, conversion was performed using calibration constants.

<small>

```python
def speed_estimation(left_speed, right_speed):
    speed_measured = (right_speed + left_speed) / 2
    speed = (speed_measured * REAL_THYMIO_SPEED) / COMMAND_MOTOR_FOR_CALIBRATION
    angular_speed_measured = (right_speed - left_speed) / 2
    angular_speed = (angular_speed_measured * REAL_THYMIO_ANGULAR_SPEED) / COMMAND_MOTOR_FOR_CALIBRATION

    return speed, angular_speed
```
</small>

Here is how the function is initialized:

<small>

```python
speed, angular_speed = speed_estimation(left_speed, right_speed)

dt = time.time() - start_time 
state_estimation, P_estimation = ex_kalman_filter(float speed, float angular_speed, bool camera_got_pos, array position_from_camera,
                                                  array state_estimation, array P_estimation, float dt)
start_time = time.time()
```
</small>

To estimate the robot's next state, the filter first predicts values based on the previous state, according to the following equations:

$$
x_{k+1} = x_{k} + v_{k} * cos(\theta _{k}) * dt
$$
$$
y_{k+1} = y_{k} + v_{k} * sin(\theta _{k}) * dt
$$
$$
\theta _{k+1} = \theta _{k} + v_{k} * cos(\theta _{k}) * dt
$$
$$
v_{k+1} = v_{k}
$$
$$
w_{k+1} = w_{k}
$$

Which are implemented as follows:

<small>

```python
## Prediciton Step, through the previous estimation
A = np.array([[1, 0, 0, np.cos(theta).item() * dt, 0],
              [0, 1, 0, np.sin(theta).item() * dt, 0],
              [0, 0, 1, 0, dt],
              [0, 0, 0, 1, 0],
              [0, 0, 0, 0, 1]])
predicted_state_estimation = np.dot(A, previous_state_estimation)
```
</small>

After measuring the standard deviation of velocity and angular rate measurements during a typical mission, it was possible to approximate their error. Assuming that half the variation is caused by the measurements and the other half by perturbations of the states by external factors (terrain, wheel slippage, etc.).

<small>

```python
STD_SPEED = 3 # mm^2/s^2
STD_ANGULAR_SPEED = 0.04 # rad^2/s^2

q_nu_translation = STD_SPEED / 2 # variance on speed state
r_nu_translation = STD_SPEED / 2 # variance on speed measurement
q_nu_rotation = STD_ANGULAR_SPEED / 2 # variance on angular speed state
r_nu_rotation = STD_ANGULAR_SPEED / 2 # variance on angular speed measurement
```

</small>

Uncertainty in state prediction is only influenced by velocity and angular speed, as these are included in the equations for position estimation. The process noise covariance matrix for prediction is as follows:

<small>

```python
Q = np.array([[0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0],
              [0, 0, 0, q_nu_translation, 0],
              [0, 0, 0, 0, q_nu_rotation]])

predicted_state_estimation_jacobian=
np.array([[1,0,-previous_state_estimation[3].item()*np.sin(theta).item()*dt, np.cos(theta).item()*dt,0], 
                                              [0, 1,previous_state_estimation[3].item()*np.cos(theta).item()*dt,np.sin(theta).item()*dt,0],
                                              [0, 0, 1, 0, dt],
                                              [0, 0, 0, 1, 0],
                                              [0, 0, 0, 0, 1]])
predicted_covariance_estimation = np.dot(predicted_state_estimation_jacobian,
                                         np.dot(previous_covariance_estimation, predicted_state_estimation_jacobian.T)) + Q
```

</small>

In a second step, the values are updated by the sensor measurements and the external camera estimation.
To update the values, the algorithm takes into account the uncertainty of each measurement through the R matrix. After representing the variation in the camera's position estimation, the horizontal and vertical error (RP) is estimated at +/- 1cm, and the orientation error (RP_ANGLE) at ≈1.5 degrees. Here's the code to update the prediction:

<small>

```python
## Update Step      
    if cam_got_pos and position_from_camera is not None:
        # camera position is available
        y = np.array([[position_from_camera[0]], [position_from_camera[1]], [position_from_camera[2]], [speed], [angular_speed]])
        H = np.identity(5)
        R = np.array([[RP, 0, 0, 0, 0],
                      [0, RP, 0, 0, 0],
                      [0, 0, RP_ANGLE, 0, 0],
                      [0, 0, 0, r_nu_translation, 0],
                      [0, 0, 0, 0, r_nu_rotation]]) # process noise covariance matrix
    else:
        # no transition, use only the speed
        y = np.array([[speed], [angular_speed]])
        H = np.array([[0, 0, 0, 1, 0], [0, 0, 0, 0, 1]])
        R = np.array([[r_nu_translation, 0], [0, r_nu_rotation]]) # process noise covariance matrix

    # measurement residual
    i = y - np.dot(H, predicted_state_estimation)
    # measurement prediction covariance
    S = np.dot(H, np.dot(predicted_covariance_estimation, H.T)) + R
    # Kalman gain (tells how much the predictions should be corrected based on the measurements)
    K = np.dot(predicted_covariance_estimation, np.dot(H.T, np.linalg.inv(S)))

    # Updated state and covariance estimate
    state_estimation = predicted_state_estimation + np.dot(K, i)    
    P_estimation = np.dot((np.identity(5) - np.dot(K, H)), predicted_covariance_estimation)
```

</small>
The function returns the estimated position and covariance, which will be used to calculate the prediction for the next estimation.




<a id="globnav"></a>
## **Global Navigation**

All the global navigation function are grouped in the folder `helpers_global`.


### A\* computation
Since we are working with a grid filled with 1 for global obstacles and 0 for free cells, we are using the same A\* algorithm as the one in the exercise on Path Planning. We made a couple changes to adapt it to the project. Note that the grid computed by the vision is taking into account the size of the robot so the obstacles are bigger than in reality. After getting, the start and goal position from the camera, we can use the  `global_final` function to get the path that is already simplified by the Douglas Peucker algorithm. It also use a VISUALIZE parameter that plot the path computed on the given grid. To naviguate further from the global obstacles, we added a cost on the gScore if the cell is near an obstacle. We computed an safety margin that multiplies the current cost (the default one is 1) that is add to gScore : 

<small>

```python
def calculate_safety_margin(neighbor, occupancy_grid):
    distance_threshold_big = 5  
    distance_threshold_small = 3
    distance_to_obstacle = distance_to_nearest_obstacle(neighbor, occupancy_grid)

    if distance_to_obstacle < distance_threshold_big:
        if distance_to_obstacle < distance_threshold_small:
            safety_margin = 2     
        else: 
            safety_margin = 1.5  
    else:
        safety_margin = 1.0  

    return safety_margin

```

</small>

where `distance_to_nearest_obstacle` gives the minimal distance to any obstacles in the grid


### Douglas-Peucker Algorithm

Since the path given by A\* is giving points on the grid and therefore is often changing in direction we simplified it by the Douglas Peucker Algorithm:

<small>

```python
def douglas_peucker(coords, epsilon):
    if len(coords) <= 2:
        return [coords[0], coords[-1]]

    dmax = 0
    index = 0
    end = len(coords) - 1
    for i in range(1, end):
        d = point_to_line_distance(coords[i], coords[0], coords[end])
        if d > dmax:
            index = i
            dmax = d

    if dmax > epsilon:
        results1 = douglas_peucker(coords[:index + 1], epsilon)
        results2 = douglas_peucker(coords[index:], epsilon)

        results = results1[:-1] + results2
    else:
        results = [coords[0], coords[end]]

    return results
```

</small>

It is a recursive algorithm that take the path in argument and give a simplified one that only keep the point with big variation in between



<a id="motion"></a>

## **Motion Control**

The motion control is implemented in the main `while(True)` loop in the Jupyter notebook file. 
Global navigation gives us the path saved in and the motion control has to guide the thymio through all the path checkpoints (`path[counter]`) until reaching the final goal (`path[end]`). The variable `counter` stores the current index of `path[]`that we aim for. When the robot’s position is close to it (=its position is within a circle of 4cm of radius around the current target), we increment counter. This enables us to target the next checkpoint even if a small local obstacle is placed on a checkpoint.

The navigation from one checkpoint to another is done by regulating `angle_error`(=orient_robot-(angle_to_next_checkpoint)). To do so, we use a P_controller for an angle_error up to $\pi$/6. Above this, the robot is rotated on the spot (left_motor_target=-right_motor_target). The P controller enables very smooth motion during the path.

In `Thymio.py` useful functions to get the information on the robot for example` get_proximity_ground_values()`, and also functions to control the movement of the thymio, like ` motorset()`

<small>

 ```python
async def get_proximity_values(client):
    # Wait for the Thymio node
    node = await client.wait_for_node()
    # Wait for the proximity sensor variables
    await node.wait_for_variables({"prox.horizontal"})
    # Get the proximity values : v: Stands for "variables" and is used to access the cached variable values.
    proximity_values = node.v.prox.horizontal
    return proximity_values[0:5]

```
</small>

<a id="demo"></a>
# Demonstration

In [ ]:
import time
import numpy as np
import cv2
import matplotlib.pyplot as plt
import opencv_jupyter_ui as jcv2

## Custom Libraries 
from src.Motion_Control import thymio as th
from src.Global_Nav import helpers_global as gn
from src.Vision import vision as vs
from src.Local_Nav import psymap as pm  
from src.Local_Nav import local_navigation as ln
from src.Filtering import filtering
from src.Motion_Control import PID



In [ ]:
## Constant 
REFRAME = True 
TS =0.01
EPSILON_ANGLE= np.pi/10
VISUALIZE = True
MAP_SHAPE_MM = (1000,700)
MAP_SHAPE_CELL = (50,35)
ROBROAD = 80
SIMPLIFY = 0.8
SAVE_VIDEO = True

In [ ]:
from tdmclient import ClientAsync
client = ClientAsync()
node = await client.wait_for_node() #_ = protected #__ = private = shouldn't access node outside of the class
await node.lock()


cap = cv2.VideoCapture(0)

if SAVE_VIDEO:
    videosaver = cv2.VideoWriter('VideoG16.avi',  cv2.VideoWriter_fourcc(*'MJPG'), 10, MAP_SHAPE_MM) 

In [ ]:
#setup
if REFRAME:
    Tmap = vs.get_warp(cap,MAP_SHAPE_MM,20,10)

ret,frame = cap.read()
if ret:
    if REFRAME:
        frame = cv2.warpPerspective(frame,Tmap,MAP_SHAPE_MM)
    fmap = vs.get_grid_fixed_map(frame,MAP_SHAPE_CELL)
    obscont = vs.get_obstacles(frame)
    print("Searching for destination...")
    while True:
        ret,frame = cap.read()
        if ret:
            if REFRAME:
                frame = cv2.warpPerspective(frame,Tmap,MAP_SHAPE_MM)
            ret,destmm = vs.get_destination(frame)
            if ret:
                dest = gn.convert_to_idx([coord / 10.0 for coord in destmm],2)
                dest[1]= 35-dest[1]
                dest = tuple(dest)
                break
            else:
                print("no destination !",end='\r')
        else:
            print("No camera !")
            break
    print("Found destination Point at {} [mm] {} [cells]".format(destmm,dest))
    print("Searching for Robot...")
    while True:
        ret,frame = cap.read()
        if ret:
            if REFRAME:
                frame = cv2.warpPerspective(frame,Tmap,MAP_SHAPE_MM)
            hls_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HLS_FULL) 
            ret,robpos,orient,pxpcm = vs.get_Robot_position_orientation(hls_frame)
            if ret:
                print("Robot found at {} [mm], {} [rad]".format(robpos,orient))
                break
        else:
            print("No camera !")
            break

start = gn.convert_to_idx(robpos,20)
start[1]= MAP_SHAPE_CELL[1]-start[1]
start = tuple(start)
path = gn.global_final(fmap,start,dest, "8N", VISUALIZE)


In [ ]:
##Main boucle with Kalman Aubin

local_obstacle = False
counter=0
cell_pos_history= []
kidnapping_state = False
check=[]
check_point_prev=np.array([0,0])
Kp = 50
spdPID = 100
start_time = time.time()

state_estimation_prev= np.array([[robpos[0]],[robpos[1]],[orient],[0],[0]])
P_estimation_prev =  np.diag([100, 100, 0.75, 10, 0.75])

while True:
    ret,frame = cap.read()
    if ret:

        if REFRAME:
            frame = cv2.warpPerspective(frame,Tmap,MAP_SHAPE_MM)

        HLS = cv2.cvtColor(frame, cv2.COLOR_BGR2HLS_FULL)

        camgotpos ,robpos,orient, scale = vs.get_Robot_position_orientation(HLS, 5)
        if camgotpos : 

            robpos[1] = MAP_SHAPE_MM[1]- robpos[1]
            orient2 = orient
            if orient <0:
                orient = orient +2*np.pi
            x_est_cam = np.array([robpos[0], robpos[1], orient])
            cell_pos_cam = np.array([x_est_cam[0]/20, x_est_cam[1]/20, orient2])
            cell_pos_history.append(cell_pos_cam)


        ground_values = await th.get_proximity_ground_values(client)
        if(ground_values[0]<300 or ground_values[1]< 300):
            print('Kidnapping detected')
            await th.stop_motor(node)
            kidnapping_state= True

        if ground_values[0]>300 and ground_values[1]>300 and kidnapping_state and camgotpos:

            kidnapping_state = False
            start = gn.convert_to_idx(robpos,20)
            start = tuple(start)
            path = gn.global_final(fmap,start,dest, "8N", VISUALIZE)
            state_estimation_prev = np.array([[robpos[0]],[robpos[1]], [orient], [0], [0]])
            P_estimation_prev =  np.diag([100, 100, 0.75, 10, 0.75])
            counter =0
            check_point_prev=np.array([0,0])
            start_time = time.time()


        if x_est_cam is not None:
            if abs(state_estimation_prev[2][0]-x_est_cam[2]) > 1:
                x_est_cam = None
        state_estimation, P_estimation, speed, angular_speed, start_time, angle = await filtering.get_position(state_estimation_prev, P_estimation_prev, start_time,camgotpos,x_est_cam, node )
        state_estimation_prev = state_estimation
        P_estimation_prev = P_estimation

        position = np.array([state_estimation[0].item(), state_estimation[1].item()])
        theta = angle
        position = position / 20.0
        position_array = np.array(position)

        state_history = np.array([position_array[0], position_array[1], theta[0]])
        check.append(state_history)

        check_point, counter = gn.next_checkpoint2(path, position, counter,local_obstacle)
            
        if not kidnapping_state:
            if np.any(check_point_prev != check_point):
                print(f"robot at {position}, grid coord {gn.convert_to_idx(position, 1)} next checkpoint at{check_point}")
                check_point_prev = check_point
                
            if abs(position[0]-path[-1][0])<1 and abs(position[1]-path[-1][1])<1:
                await th.stop_motor(node)
                break

            
            angle_error=  theta-th.compute_angle(gn.convert_to_idx(position,1), path[counter])
            if angle_error > np.pi :
                angle_error = angle_error-2*np.pi
            if angle_error < -np.pi:
                angle_error = angle_error+ 2*np.pi

            
            capthall = pm.hallucinate_map([position[0],position[1],(-orient)],obscont)
            sens = await th.get_proximity_values(client)
            if (sum(sens[i] > 1000 for i in range(0, 5)) > 0):
                local_obstacle = True

            if(local_obstacle):
                print("Entering Local navigation mode")
                await ln.local_navigation(client,node,[position[0],position[1],(-orient)],obscont)
                
                if(not sum(sens[i] > 1000 for i in range(0, 5)) > 0):
                    await th.motorset(node,100,100)
                    time.sleep(1.5)
                    local_obstacle = False
            #motor control
            else :
                if(angle_error>EPSILON_ANGLE):
                    await th.motorset(node,70,-70)
                elif (angle_error<-EPSILON_ANGLE):
                    await th.motorset(node,-70,70)
                else:
                    speed_l, speed_r = PID.PIDController(Kp,spdPID, angle_error)
                    await th.motorset(node,speed_l,speed_r)
                    

        if VISUALIZE:
            vizu = vs.visualizer(HLS)
            omap =vs.grid_fixedmap_visualizer(fmap.transpose(),MAP_SHAPE_MM)
            obsimg = cv2.merge([omap,omap,omap])
            vizu = cv2.bitwise_or(vizu,obsimg)
            vizu = vs.draw_obstacles_poly(vizu,obscont,(255,255,0),2)
            vizu = cv2.circle(vizu,destmm,20,(50,25,100),4)
            vizu = cv2.addWeighted(vizu,0.5,frame,0.5,0)
            vizu = vs.show_path(vizu,path,20,10)
            vizu = vs.show_Kalman(vizu,state_estimation,P_estimation,10)
            robpos[1] = MAP_SHAPE_MM[1] - robpos[1]
            vizu = vs.paint_robot(vizu,(0,0,200),robpos,orient,pxpcm)
            vizu = pm.hallucinate_map([robpos[0],robpos[1],(-orient)],obscont,vizu)
            videosaver.write(vizu) 
            jcv2.imshow("Map",vizu)
            if jcv2.waitKey(1) & 0xFF == ord('q'):
                await th.stop_motor(node)
                break
        else:
            print("###################################################",end='\r')
            print("pos: {},{:.2f} dest: {} fhal: {}".format(robpos,orient,destmm,capthall),end='\r')
    
    else :
        print("error : camera failure.")
        break

videosaver.release()
jcv2.destroyAllWindows()
await th.stop_motor(node)

In [ ]:

x_values = [coord[0] for coord in check]
y_values = [coord[1] for coord in check]
x_path =   [coord[0] for coord in path]
y_path =  [coord[1] for coord in path]

# Tracer le graphique x en fonction de y
plt.plot(x_values, y_values, marker='.', linestyle='-')
plt.plot(x_path, y_path, marker ='o', color = 'red')
plt.axis('equal')
plt.title('Historique de position')
plt.xlabel('x')
plt.ylabel('y')
plt.grid(True)
plt.show()


In [ ]:
await th.stop_motor(node)
node.unlock()

<a id="results"></a>
## **Results**

To conclude, the code seems to work well, the thymio reacts well to local obstacles and reaches the Goal in a smooth way, when a path exists. The only case that we did not consider is when the only path possible is blocked by local obstacles, the robot doesn’t stop and may pass over global obstacles. We struggled a lot to put all the work together and spend most of the time doing this. The filter was also hard to implement, but worked at the end and gives good approximation when the camera is hidden.